<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - Lab 3: Testing Queries for REST Server

As usual we have to tell Jupyter to allow SQL:

In [2]:
%load_ext sql

Check SQL Version

In [3]:
import sqlite3
ver = sqlite3.sqlite_version_info
print("SQLite Version {}.{}.{}".format(ver[0], ver[1], ver[2]))

SQLite Version 3.40.1


And then we import our movie database

In [4]:
%sql sqlite:///theaters.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [5]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///theaters.sqlite
Done.


[]

## SQL Query Testing Below

Write SQL code for the following tasks:

+ Give a summary of all tickets for a user, like this:
```
curl -X GET http://localhost:7007/users/alice/tickets
{
    "data": [
        {
            "date": "2021-02-22",
            "startTime": "19:00",
            "theater": "Kino",
            "title": "The Shape of Water",
            "year": 2017,
            "nbrOfTickets": 2
        },
        {
            "date": "2021-02-23",
            "startTime": "19:00",
            "theater": "Skandia",
            "title": "Moonlight",
            "year": 2016,
            "nbrOfTickets": 1
        }
    ]
}
```

In [15]:
%%sql
SELECT      *
FROM        Ticket
RIGHT JOIN  Customer
USING       (username)
WHERE       username="jacob1576"

 * sqlite:///theaters.sqlite
Done.


TicketId,PerformanceId,Username,CustomerName,UserPassword
b88887153900dbff9dd6f82eb75587d6,3,jacob1576,Jacob Krucinski,moVieAccount7428@
62390aff22b642c3ee04e2ee071acb6f,3,jacob1576,Jacob Krucinski,moVieAccount7428@
71624c294e4dae2afd6ed07772dad948,5,jacob1576,Jacob Krucinski,moVieAccount7428@
ab4949f77ac1800dbf827554a334c7e3,14,jacob1576,Jacob Krucinski,moVieAccount7428@
c213288a92d6b3b47f600bac5dabc297,17,jacob1576,Jacob Krucinski,moVieAccount7428@
e6a2dd68dbe8fe944820a2092254ea56,16,jacob1576,Jacob Krucinski,moVieAccount7428@


In [27]:
%%sql
WITH all_customer_data AS
(
    SELECT      count() AS n_tickets, PerformanceId, CustomerName, UserPassword
    FROM        Ticket
    RIGHT JOIN  Customer
    USING       (username)
    --WHERE       username="jacob1576"
    --GROUP BY    PerformanceId
    LEFT JOIN   Performance
    USING       (PerformanceId)
)

SELECT *
FROM   all_customer_data

 * sqlite:///theaters.sqlite
Done.


n_tickets,PerformanceId,CustomerName,UserPassword
16,1,John Smith,password123


In [34]:
%%sql
WITH short_customer_data AS
(
    SELECT      count() AS n_tickets, PerformanceId, CustomerName, UserPassword
    FROM        Ticket
    RIGHT JOIN  Customer
    USING       (username)
    WHERE       username="jacob1576"
    GROUP BY    PerformanceId
)

SELECT      *
FROM        short_customer_data
LEFT JOIN   Performance
USING       (PerformanceId)

 * sqlite:///theaters.sqlite
Done.


n_tickets,PerformanceId,CustomerName,UserPassword,StartTime,PerformanceDate,TheaterName,IMDBKey
2,3,Jacob Krucinski,moVieAccount7428@,17:00,2023-01-23,Artcraft Theater,tt0109830
1,5,Jacob Krucinski,moVieAccount7428@,19:10,2022-10-06,Capitol Theater,tt0468569
1,14,Jacob Krucinski,moVieAccount7428@,09:20,2023-02-04,Garneau Theater,tt0876563
1,16,Jacob Krucinski,moVieAccount7428@,12:30,2049-01-01[,Atlantic,tt1856101
1,17,Jacob Krucinski,moVieAccount7428@,21:00,2022-11-06,Astoria,tt0088247


In [49]:
%%sql
-- All in one statement
SELECT      PerformanceDate, StartTime, TheaterName, MovieTitle, ProductionYear, count() as nbrOfTickets
FROM        Ticket
LEFT JOIN   Customer
USING       (username)
LEFT JOIN   Performance
USING       (PerformanceId)
LEFT JOIN   Movie
USING       (IMDBKey)
WHERE       username="alice2002"
GROUP BY    PerformanceId

 * sqlite:///theaters.sqlite
Done.


PerformanceDate,StartTime,TheaterName,MovieTitle,ProductionYear,nbrOfTickets
2023-01-01,13:15,Artcraft Theater,Forest Gump,1994,1
2023-01-11,21:20,Booth Theater,Silence of the Lambs,1991,1
2023-02-04,19:25,Royal Cinema,Ponyo,2008,1
